In [1]:
# This notebook uses https://github.com/openai/whisper with edits to the whisper_openAI/decoding.py to generate multiple hypothesis
import sys
import os
import argparse
import re
import json
import torch
import logging
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from datasets import load_dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Dict, List, Union
from evaluate import load
from tqdm import tqdm
from datetime import datetime


In [2]:
speaker_id = "M01"  # Example value; replace with the actual Speaker ID as needed
learning_rate = 0.0001
train_batch_size = 4
eval_batch_size = 4
seed = 42
gradient_accumulation_steps = 2
optimizer = "adamw_torch"
lr_scheduler_type = "linear"
num_epochs = 20
keep_all_data = False
debug = False
repo_suffix = ""

print(f"Speaker ID: {speaker_id}")
print(f"Learning rate: {learning_rate}")
print(f"Training batch size: {train_batch_size}")
print(f"Evaluation batch size: {eval_batch_size}")
print(f"Random seed: {seed}")
print(f"Gradient accumulation steps: {gradient_accumulation_steps}")
print(f"Optimizer type: {optimizer}")
print(f"Learning rate scheduler type: {lr_scheduler_type}")
print(f"Number of epochs: {num_epochs}")
print(f"Keep all data: {keep_all_data}")
print(f"Debug mode: {debug}")
print(f"Repository suffix: {repo_suffix}")
test_speaker = speaker_id

Speaker ID: M01
Learning rate: 0.0001
Training batch size: 4
Evaluation batch size: 4
Random seed: 42
Gradient accumulation steps: 2
Optimizer type: adamw_torch
Learning rate scheduler type: linear
Number of epochs: 20
Keep all data: False
Debug mode: False
Repository suffix: 


In [3]:
import os
import logging

# Define the path to the CSV file
torgo_csv_path = "data_preparation/torgo.csv"

# Check if the path exists and is a file
if os.path.exists(torgo_csv_path) and os.path.isfile(torgo_csv_path):
    print("The CSV file exists.")
else:
    print("The CSV file does not exist.")

torgo_dataset_path = '/work/van-speech-nlp/data/torgo'
torgo_dataset_dir_path = torgo_dataset_path + \
        '/' if torgo_dataset_path[-1] != '/' else torgo_dataset_path
output_path = 'output'
print(f'torgo_dataset_path: {torgo_dataset_path}')
print(f'torgo_dataset_dir_path: {torgo_dataset_dir_path}')

repo_name = f'torgo_tiny_finetune_{test_speaker}{repo_suffix}'
repo_path = f'jindaxz/{repo_name}'

# Path to save model / checkpoints{repo_name}'
model_local_path = output_path + '/model/' + repo_name

pretrained_model_name = "openai/whisper-tiny"

The CSV file does not exist.
torgo_dataset_path: /work/van-speech-nlp/data/torgo
torgo_dataset_dir_path: /work/van-speech-nlp/data/torgo/


In [4]:
%cd ..

/work/van-speech-nlp/jindaznb/jslpnb/mllm_expriments/Whispering-LLaMA


In [5]:
import numpy
# Renamed the Whisepr repo (https://github.com/openai/whisper) with the changed decoding.py file as whisper_openAI
import whisper_openAI.whisper as whisper
import torch
import tqdm
model, _ = whisper.load_model("large-v2") # you can change the whisper model here to largev2 or large to swap the  model.

In [6]:
model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-31): 32 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=1280, out_features=1280, bias=True)
          (key): Linear(in_features=1280, out_features=1280, bias=False)
          (value): Linear(in_features=1280, out_features=1280, bias=True)
          (out): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (attn_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=1280, out_features=5120, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=5120, out_features=1280, bias=True)
        )
        (mlp_ln): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((

In [7]:
data_df = pd.read_csv(torgo_csv_path)
dataset_csv = load_dataset('csv', data_files=torgo_csv_path)

# Check if the following columns exist in the dataset ['session', 'audio', 'text', 'speaker_id']
expected_columns = ['session', 'audio', 'text', 'speaker_id']
not_found_columns = []
for column in expected_columns:
    if column not in dataset_csv['train'].column_names:
        not_found_columns.append(column)

if len(not_found_columns) > 0:
    logging.error(
        "The following columns are not found in the dataset:" + " [" + ", ".join(not_found_columns) + "]")
    sys.exit(1)

In [8]:
logging.info(
    "Splitting the dataset into training / validation / test sets...")

# Extract the unique speakers in the dataset
speakers = data_df['speaker_id'].unique()

logging.info("Unique speakers found in the dataset:")
logging.info(str(speakers) + '\n')

if test_speaker not in speakers:
    logging.error("Test Speaker not found in the dataset.")
    sys.exit(1)

valid_speaker = 'F03' if test_speaker != 'F03' else 'F04'
train_speaker = [s for s in speakers if s not in [
    test_speaker, valid_speaker]]

torgo_dataset = DatasetDict()
torgo_dataset['train'] = dataset_csv['train'].filter(
    lambda x: x in train_speaker, input_columns=['speaker_id'])
torgo_dataset['validation'] = dataset_csv['train'].filter(
    lambda x: x == valid_speaker, input_columns=['speaker_id'])
torgo_dataset['test'] = dataset_csv['train'].filter(
    lambda x: x == test_speaker, input_columns=['speaker_id'])

In [9]:
original_data_count = {'train': len(torgo_dataset['train']), 'validation': len(
    torgo_dataset['validation']), 'test': len(torgo_dataset['test'])}

if not keep_all_data:
    # Update the three dataset splits (if ['test_data'] == 1, keep in test, if ['test_data'] == 0, keep in train and validation)
    torgo_dataset['train'] = torgo_dataset['train'].filter(
        lambda x: x['test_data'] == 0)
    torgo_dataset['validation'] = torgo_dataset['validation'].filter(
        lambda x: x['test_data'] == 0)
    torgo_dataset['test'] = torgo_dataset['test'].filter(
        lambda x: x['test_data'] == 1)

    # Drop the 'test_data' column
    torgo_dataset['train'] = torgo_dataset['train'].remove_columns([
                                                                   'test_data'])
    torgo_dataset['validation'] = torgo_dataset['validation'].remove_columns([
                                                                             'test_data'])
    torgo_dataset['test'] = torgo_dataset['test'].remove_columns([
                                                                 'test_data'])
    logging.info(
        f"After removal of repeated prompts, the number of data in each dataset is:")
    logging.info(
        f'Train:       {len(torgo_dataset["train"])}/{original_data_count["train"]} ({len(torgo_dataset["train"]) * 100 // original_data_count["train"]}%)')
    logging.info(
        f'Validation:  {len(torgo_dataset["validation"])}/{original_data_count["validation"]} ({len(torgo_dataset["validation"]) * 100 // original_data_count["validation"]}%)')
    logging.info(
        f'Test:        {len(torgo_dataset["test"])}/{original_data_count["test"]} ({len(torgo_dataset["test"]) * 100 // original_data_count["test"]}%)\n')

In [10]:
# Remove special characters from the text
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\`\�0-9]'


def remove_special_characters(batch):
    batch['text'] = re.sub(chars_to_ignore_regex,
                           ' ', batch['text']).lower()
    return batch

torgo_dataset = torgo_dataset.map(remove_special_characters)

In [11]:

print(torgo_dataset['train'][2]['text'])

meat


In [12]:
import whisper_openAI.whisper as whisper

# Function to process each audio file
def process_audio(file_path):
    # Load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(file_path)
    audio = whisper.pad_or_trim(audio)
    
    # Make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    # Decode the audio
    options = whisper.DecodingOptions(fp16=False, without_timestamps=True, language='english')
    result, _ = whisper.decode(model, mel, options)
    
    result_text = ''.join(result)
    
    # Print the recognized text
    print(result_text)
    return result_text

# Initialize the results and ground truth lists
recognized_texts = []
ground_truth_texts = []

def normalize_text(text):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\`\�0-9]'
    text = re.sub(chars_to_ignore_regex, ' ', text).lower()
    return text
    
# Iterate over the dataset
for i in range(len(torgo_dataset['test'])):
    # Get the file path and ground truth from the dataset
    file_path = torgo_dataset['test'][i]['audio']
    ground_truth = torgo_dataset['test'][i]['text']
    
    # Process the audio file
    recognized_text = process_audio(file_path)
    recognized_text = normalize_text(recognized_text)
    ground_truth = normalize_text(ground_truth)
    
    # Append the results to the lists
    recognized_texts.append(recognized_text)
    ground_truth_texts.append(ground_truth)
    
    # Print the recognized text
    print(f"text {i+1}/{len(torgo_dataset['test'])}: {recognized_text}")
    print(f"Ground truth: {ground_truth}")
    print()

when a pig is buried inside a bears crack and quivers a dreadful
text 1/407: when a pig is buried inside a bears crack and quivers a dreadful
Ground truth: when he speaks his voice is just a bit cracked and quivers a trifle

trade
text 2/407: trade
Ground truth: trait

trouble
text 3/407: trouble
Ground truth: trouble

see
text 4/407: see
Ground truth: fee

raid
text 5/407: raid
Ground truth: raid

a gap in the winter when the wool or not our eyes permit
text 6/407: a gap in the winter when the wool or not our eyes permit
Ground truth: except in the winter when the ooze or snow or ice prevents

and unfiltering beer drinks to each time
text 7/407: and unfiltering beer drinks to each time
Ground truth: a long flowing beard clings to his chin

a long flowing beer cling to each chin
text 8/407: a long flowing beer cling to each chin
Ground truth: a long flowing beard clings to his chin

fear
text 9/407: fear
Ground truth: fair

beard
text 10/407: beard
Ground truth: bit

bubble
text 11/407

In [13]:
from jiwer import wer

# Calculate WER for each recognized text against the ground truth
wer_scores = [wer(gt, rt) for gt, rt in zip(ground_truth_texts, recognized_texts)]

# Print the average WER
average_wer = sum(wer_scores) / len(wer_scores)
print(f"Average WER: {average_wer}")

# Optional: Save the recognized texts and ground truths to files
with open('recognized_texts.txt', 'w') as f:
    for text in recognized_texts:
        f.write(f"{text}\n")

with open('ground_truth_texts.txt', 'w') as f:
    for text in ground_truth_texts:
        f.write(f"{text}\n")

Average WER: 0.9603567194476285


In [14]:
print()

In [15]:
recognized_texts

['when a pig is buried inside a bears crack and quivers a dreadful',
 'trade',
 'trouble',
 'see',
 'raid',
 'a gap in the winter when the wool or not our eyes permit',
 'and unfiltering beer drinks to each time',
 'a long flowing beer cling to each chin',
 'fear',
 'beard',
 'bubble',
 'while he is nearly    years old',
 'brad',
 'danger',
 'no',
 'yet either links as swively as ever',
 'jagged',
 'and dabble',
 'mike',
 'yeah',
 'worm ',
 'dear',
 'i dont know',
 'err',
 'dont ask me to carry an oily rag like a bat',
 'stumble',
 'bark',
 'you need to know all about my grandfather',
 'good',
 'giving those who observe him a better understanding of the atmosphere he brings',
 'yip',
 'rate',
 'zip',
 'trade',
 'no ',
 'the end youre talking to an increasingly white one oh you',
 'oh',
 'bye',
 'rape',
 'beda',
 'i will take a long walk in the open air indeed',
 'or',
 'today',
 'no',
 'its right in the uterus carefully and we rest upon our small organs',
 'spark again and again',
 'si